In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
sudo apt-get install -f

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,443 kB]
Get:14 http://archive.ubuntu.com/ubunt

In [ ]:
!pip install chromedriver-py
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import sys
import logging
from google.colab import files
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
import pandas as pd
import json
import pprint
from chromedriver_py import binary_path
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
chrome_options = webdriver.ChromeOptions()  # Initialize Chrome options
chrome_options.add_argument('--headless')  # Run Chrome in headless mode - In our local notebook we can remove the `--headless` option.
chrome_options.add_argument('--no-sandbox')  # Disable Chrome’s default sandboxing behavior
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument("window-size=1900,800")  # Set the window size for the browser
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")  # Set a custom user agent

In [ ]:
service = Service(executable_path=binary_path)  # Create a Service object with the path to the ChromeDriver executable
wd = webdriver.Chrome(service=service, options=chrome_options)  # Initialize the WebDriver with the specified service and options

In [ ]:
def show_page():
    wd.save_screenshot('screenshot.png')

    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg

    img = mpimg.imread('/content/screenshot.png')
    plt.figure(figsize=(20, 10))
    imgplot = plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
wd.get('https://www.corriere.it/account/login?landing=https://www.corriere.it/')

In [ ]:
show_page()

NameError: name 'show_page' is not defined

In [ ]:
# Effettuo il log-in

login = wd.find_elements(By.CSS_SELECTOR, "input.style__StyledInput-sc-14879tp-1.Itulx")[0]
login.send_keys("francesco.bogazzi1994@gmail.com")

password = wd.find_elements(By.CSS_SELECTOR, "input.style__StyledInput-sc-14879tp-1.Itulx")[1]
password.send_keys("ProgettoMaster24")

login.send_keys(Keys.RETURN)

In [ ]:
# Accetto i cookies

wd.find_elements(By.CSS_SELECTOR, "button.privacy-cp-footer__button_cmp.privacy-cp-footer__button_cmp--big.privacy-cp-footer__button_cmp--main")[2].send_keys(Keys.RETURN)

In [ ]:
# Accedo all'area "Sezioni" e cerco "siccità"

wd.find_elements(By.CSS_SELECTOR, "a.navbar-link")[0].send_keys(Keys.RETURN)

cerca = wd.find_elements(By.CSS_SELECTOR, "input.search-input.js-search-input")[0]
cerca.send_keys("siccità")

cerca.send_keys(Keys.RETURN)

In [ ]:
seen_articles = []
article_date = []
article_author = []
article_text = []
article_bold_words = []
page_number = 1


wd.get(f"https://www.corriere.it/ricerca/?refresh_ce=&q=siccit%25C3%25A0&page={page_number}")
list_articles = wd.find_elements(By.CSS_SELECTOR, "h3.title-art-hp.is-medium.is-line-h-11")

# Estraggo tutti gli articoli sul tema siccità del 2023 e 2024.

while list_articles != [] and page_number <= 50:
  seen_articles_url = []
  for article in list_articles:
    article_title  = article.text
    seen_articles.append(article_title)
    url = article.find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("href")
    seen_articles_url.append(url)

  for URL in seen_articles_url:
    wd.set_page_load_timeout(1000)
    wd.get(URL)

    # Estraggo il testo per ogni articolo

    article_text_draft = []
    len_articles = len(wd.find_elements(By.CSS_SELECTOR, "p.chapter-paragraph"))
    for j in range(0,len_articles):
      paragraph = wd.find_elements(By.CSS_SELECTOR, "p.chapter-paragraph")[j].text
      article_text_draft.append(paragraph)
    article_text_single_article = " ".join(article_text_draft)
    article_text.append(article_text_single_article)

    # Estraggo le parole in grassetto

    list_bold_words = []
    number_of_bold_words = len(wd.find_elements(By.TAG_NAME,"b"))
    for l in range(0,number_of_bold_words):
      bold_word = wd.find_elements(By.TAG_NAME, "b")[l].text
      list_bold_words.append(bold_word)
    try:
      article_bold_words.append(list_bold_words)
    except:
      list_bold_words = ""
      article_bold_words.append(list_bold_words)

    # Per ogni articolo estraggo anche la data e l'autore

    try:
      date = wd.find_elements(By.CSS_SELECTOR, "p.is-last-update")[0].text
      article_date.append(date)
    except:
      try:
        date = wd.find_elements(By.CSS_SELECTOR, "div.article-date-place")[0].text
        article_date.append(date)
      except:
        try:
          date = wd.find_elements(By.CSS_SELECTOR, "span.article-date")[0].text
          article_date.append(date)
        except:
          try:
            date = wd.find_elements(By.CSS_SELECTOR, "span.date-writer")[0].text
            article_date.append(date)
          except:
            date = ""
            article_date.append(date)
    try:
      author = wd.find_elements(By.CSS_SELECTOR, "span.writer")[0].text
      article_author.append(author)
    except:
      try:
        author = wd.find_elements(By.CSS_SELECTOR, "a.show-user")[0].text
        article_author.append(author)
      except:
        try:
          author = wd.find_elements(By.CSS_SELECTOR, "strong.signature")[0].text
          article_author.append(author)
        except:
          try:
            author = wd.find_elements(By.CSS_SELECTOR, "span.writer.is-mr-l-4")[0].text
            article_author.append(author)
          except:
            try:
              author = wd.find_elements(By.CSS_SELECTOR, "p.author-art.is-xsmall.has-text-mine-shaft2")[0].text
              article_author.append(author)
            except:
              author = ""
              article_author.append(author)

    wd.back()

  print(page_number)
  page_number += 1

  wd.get(f"https://www.corriere.it/ricerca/?refresh_ce=&q=siccit%25C3%25A0&page={page_number}")
  list_articles = wd.find_elements(By.CSS_SELECTOR, "h3.title-art-hp.is-medium.is-line-h-11")


41
42
43
44
45
46
47
48
49
50


In [ ]:
seen_articles, article_date, article_author, article_text, article_bold_words

(['Siccità in Val Vertova, due sorgenti su tre sono asciutte: «Ci saranno problemi anche sulla rete potabile»',
  '«Siccità, terra come polvere»: a Bergamo gli agricoltori pensano a colture senz’acqua',
  'Siccità, per salvare l’estate servono 50 giorni di pioggia',
  'Allarme siccità: ecco perché è difficile «creare» l’acqua in laboratorio. I nodi e i progetti',
  'Allarme siccità, preoccupa il fiume Po in secca: la diretta video dal Ponte della Becca',
  'Siccità, il Po e il Lago di Como «in ritirata» come a Ferragosto: «Un disastro per le coltivazioni»',
  'Venezia, canali in secca nel clou del Carnevale. Gondole incagliate, ambulanze in difficoltà: «Sanitari a piedi con i malati in braccio»',
  'Edison, ricavi triplicati oltre 30 miliardi ma utile crollato del 63%: perché è colpa delle tasse',
  'Venezia a secco, canali e rii ora si svuotano. E il livello scenderà ancora',
  "Po e Garda, livelli d'acqua ai minimi storici: pronto il piano di emergenza",
  'Il Garda chiude i rubinett

In [ ]:
# Data dictionary
data = {
    "Title": seen_articles,
    "Date": article_date,
    "Author": article_author,
    "Text": article_text,
    "Bold_words": article_bold_words,
}

# Create DataFrame
df_Corriere_della_sera = pd.DataFrame(data)

# File path for saving
file_path = "Corriere_della_sera.xlsx"

# Save DataFrame to CSV
df_Corriere_della_sera.to_excel(file_path, index=False)

# Download the file in Google Colab
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>